<a href="https://colab.research.google.com/github/IshtiSikder/Optimal-allocation-of-rescue-teams-for-hostage-situations/blob/test/toy_prob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CTMC MDP model for rescue operational system!!!

prob <- function(opSet = policySet, t, lam, eps, mu) {
  opTRM = matrix(0, nrow = t+1, ncol = t+1)
  for (i in 2:t) {
    opTRM[i, i-1] = sum(i*eps, opSet[i]*mu)
    opTRM[i, i+1] = lam
  }
  opTRM[1, 2] = lam
  opTRM[t+1, t] = sum(t*eps, opSet[t+1]*mu)

  k = t+1
  steady_prob <- matrix(0, nrow=1, ncol=k)
  state <- matrix(0, nrow=1, ncol=k-1)
  for (i in 1:(k-1)) {
    if (i-2 < 0) {
      state[i] <- (lam/opTRM[i+1,i])
    } else {
      state[i] <- state[i-1]*(lam/opTRM[i+1,i])
    }
  }

  steady_prob[1] <- (1+sum(state))^-1
  for (i in 2:k) {
    steady_prob[i] <- state[i-1]*steady_prob[1]
  }
  return(steady_prob)
}




# Check function to determine new policy is same with current or not
checkPolicy = function(policySet, policySetnew) {
  n = 0
  for (i in policySet == policySetnew) {
    if (i == 'FALSE') {
      n = 1
    }
  }
  if (n == 0) {
    return("TRUE")
  } else {
    return("FALSE")
  }
}


# Define MDP function (for basic policy)
MDP = function(lam, mu, eps, numRescue, numTerror, costT, RC, mult) {
  r = numRescue
  t = numTerror

  #calculating costR

  costR = mult*costT*(mu+eps)


  #########################################################
  #EDITS: 22ND FEB,2023
  #NEW SERVICE RATES STRUCTURE, ADD ORIGINAL ONE AS WELL
  aijMatrix = array(0, dim = c(t+1,t+1,r+1))
  for (k in 1:(r+1)) {
    for (i in 1:t) {
      aijMatrix[i,i+1,k] <- lam
      if (i > 1) {
        #if (k==4){
          #if(i==4){
            #cat('min(k-1,i-1)',min(k-1,i-1))}}
        aijMatrix[i,i-1,k] <- sum(min(k-1,i-1)*mu+(i-1)*eps)
      }
    }
    aijMatrix[t+1,t,k] <- (min(k-1,t)*mu+t*eps)
    aijMatrix[1,2,k] <- lam
}

  for (k in 1:(r+1)) {
    for (i in 1:(t+1)) {

      aijMatrix[i,i,k] <- -sum(aijMatrix[i,,k])
    }
  }
  #########################################################


  # Design the MDP algorithm
  # Value determination
  policy = array(0, dim = c(1,t+1))
  policyOld = array(0, dim = c(1,t+1))
  policyNew = array(1, dim=c(1,t+1)) # initially set all decision is 1 in first policy
  policyTrack = array(2, dim=c(1,t+1))
  iteration = 0
  gainMatrix = c()

  while (checkPolicy(policyOld, policyNew) == 'FALSE') {
    policyOld = policyNew
    policy = policyNew
    # Design the transition matrix regarding policy
    A = array(0, dim = c(t+1,t+1))
    for (i in 1:length(policy)) {
      A[i,] = aijMatrix[i,,(policy[i])]
    }

    # Design the cost matrix regarding policy

    q = array(0,dim=c(t+1,1))
    for (i in 1:length(policy)) {

      if (RC == 'L'){
      cost = (policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'SR'){
      cost = sqrt(policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'Q'){
      cost = ((policy[i]-1)^2)*costR + (i-1)*eps*costT
      }

      q[i,1] = cost
    }


    newA = cbind(array(1, dim=c(t+1,1)), -A[,1:t])
    solution = solve(newA) %*% q
    gain = solution[1] # First row of solution is gain value
    gainMatrix = append(gainMatrix, gain)

    # Policy improvement
    newsolution = array(0, dim=c(t+1,1))
    for (i in 1:t) {
      newsolution[i] = solution[i+1]
    }


    newsolution[t+1] = 0 # Set last v value as 0 to calcultate the test quantity in policy improvement
    improveMatrix = array(0, dim=c(t+1,r+1))
    cost = array(0,dim=c(t+1,r+1))

    for (i in 1:(t+1)) {
      for (j in 1:(r+1)) {
        # Cost when we have j rescue operation and i terrorists

        if (RC == 'L'){
      cost[i,j] = (j-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'SR'){
      cost[i,j] = sqrt(j-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'Q'){
      cost[i,j] = ((j-1)^2)*costR + (i-1)*eps*costT
      }

        improveMatrix[i,j] = cost[i,j] + aijMatrix[i,,j] %*% newsolution

      }
    }

    # pick the minimum value among the decision
    policyNew = array(1, dim = c(1,t+1))
    for (i in 1:length(policyNew)) {
      policyNew[i] = which.min(improveMatrix[i,])
    }
    iteration = iteration + 1
    policyTrack = rbind(policyTrack, policyNew)
    policyNew
  }
  for (i in 1:length(policyNew)) {
    policyNew[i] = policyNew[i] - 1
  }
  for (i in 1:nrow(policyTrack)) {
    for (j in 1:ncol(policyTrack)) {
      policyTrack[i,j] = policyTrack[i,j] - 1
    }

  steady = prob(opSet = policyNew, t=numTerror, lam = lam, eps = eps, mu = mu)

  gain = solution[1]

  }


   #return(list(policyNew,improveMatrix,cost,aijMatrix,newsolution))

   return (policyNew)

 }

In [2]:
MDP(lam=0.2763, mu=0.2829, eps=0.1718, numRescue=10, numTerror=10, costT=1, RC='L', mult=0.5316190142082907188481)

0,1,2,3,4,5,6,7,8,9,0


In [3]:
mult_test = function(cost_terror,rc,low_lim,up_lim,incr) {

prev_result = array(100,dim=c(1,11))

for (val in seq(low_lim,up_lim, by = incr)){

        current_result = MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10, costT=cost_terror, RC=rc, mult=val)


        if (!(identical(current_result,prev_result))) {

          cat('\n\n','mult:')
          print(val,digits=22)
          cat('RC:',rc,',','\n\n')
          cat('optimal policy:\n')
          print(current_result)


          prev_result = current_result

        }

    }

}

In [6]:
#lowest possible increment possible in R is 0.00000000001, or 10^-11



mult_test(1,'L',0,3,0.001)






 mult:[1] 0
RC: L , 

optimal policy:
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
[1,]    0    1    2    3    4    5    6    7    8     9    10


 mult:[1] 0.5320000000000000284217
RC: L , 

optimal policy:
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
[1,]    0    1    2    3    4    5    6    7    8     9     0


 mult:[1] 0.6069999999999999840128
RC: L , 

optimal policy:
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
[1,]    0    1    2    3    4    5    6    7    8     0     0


 mult:[1] 0.6199999999999999955591
RC: L , 

optimal policy:
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
[1,]    0    1    2    3    4    5    6    7    0     0     0


 mult:[1] 0.6219999999999999973355
RC: L , 

optimal policy:
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
[1,]    0    1    2    3    4    5    0    0    0     0     0


 mult:[1] 0.6229999999999999982236
RC: L , 

optimal policy:
     [,1] [,2] [,3] [,4] 

In [5]:
 #THE FOLLOWING CAN BE AN ALTERNATIVE TO THE mult_test
 #tried R's apply function to avoid the delay in for loops. This doesn't work properly.
 #For some reason, runif is not creating the assigned number of random values, but less.


 mult_vals = runif(10000000,0,0.5320000000000000284217)

  mult_vals = sort(mult_matrix, decreasing=FALSE)

  mult_matrix = matrix(mult_vals,nrow=length(mult_vals))

  MDP_mult <- function(val){
  MDP(lam=0.2763, mu=0.2829, eps=0.1718, numRescue=10, numTerror=10, costT=1, RC='L', mult=val)
}

policy_matrix <- apply(mult_matrix, 1, MDP_mult)

print(unique(policy_matrix,MARGIN = 2))

ERROR: ignored

In [28]:
#prob_search algo
#the if-else statements don't seem to be working logically for the actual prob.NEED TO FIX!
#check documentation for setequal(). You are probably applying it incorrectly.

#update1 (27TH JUNE): Rather than setequal, use double equals between arrays to compare, and then use all() function to check if all same index elements are equal between both arrays.

#update2 (27TH JUNE) : For the actual prob, guess one possible intermediary policy between two found, and run the following to get close enough upper and lower bounds.
#Then using those bounds, use the mult_test alternative to check for a large number of values in between bounds. Might work, tho clunky af.

#update3(28th JUNE) :

#1. FOR MDP(lam=0.2763, mu=0.2829, eps=0.1718, numRescue=10, numTerror=10, costT=1, RC='L', mult=val), val = 0.5312087939634377864451 yields (0,1,2,.......,9)
#2. Need to figure out how does R handle large numbers, as in how many digits does it consider after the decimal point when doing any operation
#3. If possible, convert the following to apply method, cause for loops are still clunky when the lower and upper bounds get closer together.
#4. For 2, check this link: https://floating-point-gui.de/basic/

#update3(2nd July):

#1. R is precise roughly till 16 digits after the decimal point. Rest is noise.
#2. Check algo captured on iphone camera, try it out.

low = 0.5312087939634377864451
up = 0.5312087939634614341955

#go = 0

prev = 'a'

for (n in 1:3){

val = runif(1,low,up)


result = MDP(lam=0.2763, mu=0.2829, eps=0.1718, numRescue=10, numTerror=10, costT=1, RC='L', mult=val)

array_1 = array(c(0,1,2,3,4,5,6,7,8,9,10))
array_2 = array(c(0,1,2,3,4,5,6,7,8,9,0))



if (all(result == matrix(array_1,nrow=1,ncol=length(array_1)))){
  low = val
  if (!prev == 'a'){
    up = prev
  }
  next
}

else if (all(result == matrix(array_2,nrow=1,ncol=length(array_2)))){
  if (!prev == 'a'){
    low = prev
  }
  up = val
  next
}

else {
  cat('inter_MULT: ')
  print(val,digits=16)
  cat('\ninter_RESULT: ',result)
  cat('\n\n')
  #go = 1
 next

}



# #if (setequal(result == array(c(0,1,2,3,4,5,6,7,8,9,9)))){
# #  cat('JACKPOT:\n\n')
# #  print(val,digits=22)
# #  cat('result: ',result)
# #  cat('\n\n')
#   go = 1
#   break
# }

# # else {
# #   if (setequal(result,array(c(0,1,2,3,4,5,6,7,8,9,10)))) {
# #   low = val
# #   if (!prev == 'a'){
# #     up = prev
# #   }
# #   next
# # }

# else if (setequal(result,array(c(0,1,2,3,4,5,6,7,8,9,0)))){
#   if (!prev == 'a'){
#     low = prev
#   }
#   up = val
#   next
# }

# else {
#   next
# }

#}

prev = val


}

#if (go == 0) {
cat('low: ')
print(low,digits=16)
cat('\n\n')

cat('val: ')
print(val,digits=16)
cat('\n\n')

cat('up: ')
print(up,digits=)
cat('\n\n')

cat('result',result)
#}

low: [1] 0.5312087939634378


val: [1] 0.5312087939634412


up: [1] 0.5312087939634412


result 0 1 2 3 4 5 6 7 8 9 0

In [30]:
MDP_mult <- function(val){
  MDP(lam=0.2763, mu=0.2829, eps=0.1718, numRescue=10, numTerror=10, costT=1, RC='L', mult=val)
}

In [35]:
vals = unique(sort(runif(10,low,up),decreasing = FALSE))
mat = matrix(vals,nrow=length(vals))
policy <- apply(mat, 1, MDP_mult)
print(policy)

Due to the computational limitations in R, to run large and complicated while/for loops are next to impossible. As such, to find the mult value that yields our desired policy for this toy problem (0,1,1) under any cost structure, we can follow these steps:

1. First run the mult_test function to get tentative mult values for (0,1,2) and (0,1,0) while (0,1,1) doesn't yield.

2. Next, run prob_search algo with mult_(0,1,2) as low and mult_(0,1,0) as up value. While choosing the number of iterations under the for loop running for this algo, start from an arbitrary value (i.e 100) and lower the value by i.e 10 if code doesn't finish running in i.e 1 second. Keep doing this till your iteration value has been lowered to 1. The final low and up values generated should be quite closer to each other now.

3. From this point, I am kind of blank.

Actual problem has more than one intermediary policy. How to track them all without an exhaustive search, and with prob search ? I can track assuming there are as many intermediaries as I think there should be, but what happens when I am wrong ?